<a href="https://colab.research.google.com/github/yukinaga/generative_ai_programming_learning/blob/main/section_2/python_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 数値の変数を定義
number = 15
pi = 3.14159
number = 1.5

# 文字列の変数を定義
greeting = "Hello, Python world!"

# ブール値の変数を定義
is_learning = False

# 変数の内容を出力
print("整数:", number)
print("浮動小数点数:", pi)
print("挨拶:", greeting)
print("学習中か？:", is_learning)

整数: 1.5
浮動小数点数: 3.14159
挨拶: Hello, Python world!
学習中か？: False


In [ ]:
number = int("123")

In [ ]:
import random

def get_computer_choice():
    """コンピュータの選択をランダムに決定する関数"""
    return random.choice(['協力', '争う'])

def get_player_choice():
    """プレイヤーに入力を求め、選択を受け取る関数"""
    choice = input("協力しますか？争いますか？（協力/争う）: ")
    while choice not in ['協力', '争う']:
        choice = input("無効な入力です。協力または争うを正確に入力してください（協力/争う）: ")
    return choice

def determine_winner(player, computer):
    """プレイヤーとコンピュータの選択を比較し、それぞれのスコアを計算する関数"""
    if player == '協力' and computer == '協力':
        return 2, 2
    elif player == '協力' and computer == '争う':
        return 0, 3
    elif player == '争う' and computer == '協力':
        return 3, 0
    else:
        return 1, 1

def game():
    """ゲームのメインループを実行する関数"""
    player_score = 0
    computer_score = 0
    rounds = 5

    for _ in range(rounds):
        player_choice = get_player_choice()
        computer_choice = get_computer_choice()
        print(f"あなたの選択: {player_choice}, コンピュータの選択: {computer_choice}")

        player_points, computer_points = determine_winner(player_choice, computer_choice)
        player_score += player_points
        computer_score += computer_points

        print(f"このラウンドの結果: あなたの得点 {player_points}, コンピュータの得点 {computer_points}")
        print(f"現在のスコア: あなた {player_score} - コンピュータ {computer_score}")

    if player_score > computer_score:
        print("おめでとうございます！あなたの勝ちです！")
    elif player_score < computer_score:
        print("残念、コンピュータの勝ちです。")
    else:
        print("引き分けです！")

if __name__ == "__main__":
    game()


協力しますか？争いますか？（協力/争う）: 協力
あなたの選択: 協力, コンピュータの選択: 協力
このラウンドの結果: あなたの得点 2, コンピュータの得点 2
現在のスコア: あなた 2 - コンピュータ 2
協力しますか？争いますか？（協力/争う）: 協力
あなたの選択: 協力, コンピュータの選択: 争う
このラウンドの結果: あなたの得点 0, コンピュータの得点 3
現在のスコア: あなた 2 - コンピュータ 5
協力しますか？争いますか？（協力/争う）: 争う
あなたの選択: 争う, コンピュータの選択: 協力
このラウンドの結果: あなたの得点 3, コンピュータの得点 0
現在のスコア: あなた 5 - コンピュータ 5
協力しますか？争いますか？（協力/争う）: 争う
あなたの選択: 争う, コンピュータの選択: 争う
このラウンドの結果: あなたの得点 1, コンピュータの得点 1
現在のスコア: あなた 6 - コンピュータ 6
協力しますか？争いますか？（協力/争う）: 争う
あなたの選択: 争う, コンピュータの選択: 協力
このラウンドの結果: あなたの得点 3, コンピュータの得点 0
現在のスコア: あなた 9 - コンピュータ 6
おめでとうございます！あなたの勝ちです！


In [ ]:
# 必要なライブラリをインポートする
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# GPUの使用が可能かどうかをチェックする
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MNISTデータセットをダウンロードし、トレーニングとテスト用に準備する
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Imageまたはnumpy.ndarrayをTensorに変換する
    transforms.Normalize((0.5,), (0.5,))  # データの正規化を行う
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 手書き文字認識のためのニューラルネットワークモデルを定義する
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()  # 画像を1次元ベクトルに変換する
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# モデルと損失関数、オプティマイザを設定する
model = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# ネットワークのトレーニングを行う関数
def train(model, trainloader, criterion, optimizer, device):
    model.train()
    for batch, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# テストデータでネットワークを評価する関数
def evaluate(model, testloader, device):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# モデルのトレーニング
train(model, trainloader, criterion, optimizer, device)

# モデルの評価
accuracy = evaluate(model, testloader, device)
print(f'Accuracy: {accuracy}%')


100%|██████████| 9912422/9912422 [00:00<00:00, 109145519.39it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 48531928.62it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 32034477.89it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12259027.52it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Accuracy: 94.57%


In [ ]:
# PyTorchとその他の必要なライブラリをインポート
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch import optim
import torch.nn.functional as F

# GPUが使える場合はGPUを使う
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 画像の前処理を定義
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 訓練データ
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# テストデータ
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# クラス名
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

# 損失関数とオプティマイザー
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# モデルの訓練
for epoch in range(10):  # データセットを複数回繰り返して学習します
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

100%|██████████| 170498071/170498071 [00:04<00:00, 41760618.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[1,  2000] loss: 2.260
[1,  4000] loss: 1.953
[1,  6000] loss: 1.700
[1,  8000] loss: 1.603
[1, 10000] loss: 1.550
[1, 12000] loss: 1.485
[2,  2000] loss: 1.418
[2,  4000] loss: 1.388
[2,  6000] loss: 1.366
[2,  8000] loss: 1.325
[2, 10000] loss: 1.318
[2, 12000] loss: 1.288
[3,  2000] loss: 1.218
[3,  4000] loss: 1.205
[3,  6000] loss: 1.212
[3,  8000] loss: 1.213
[3, 10000] loss: 1.211
[3, 12000] loss: 1.170
[4,  2000] loss: 1.104
[4,  4000] loss: 1.112
[4,  6000] loss: 1.101
[4,  8000] loss: 1.114
[4, 10000] loss: 1.109
[4, 12000] loss: 1.111
[5,  2000] loss: 1.020
[5,  4000] loss: 1.017
[5,  6000] loss: 1.066
[5,  8000] loss: 1.039
[5, 10000] loss: 1.060
[5, 12000] loss: 1.034
[6,  2000] loss: 0.961
[6,  4000] loss: 0.995
[6,  6000] loss: 0.983
[6,  8000] loss: 0.999
[6, 10000] loss: 0.992
[6, 12000] loss: 0.972
[7,  2000] loss: 0.924
[7,  4000] loss: 0.906
[7,  6000] loss: 0.950
[7,  8000] loss: 0.944
[7, 10000] loss: 0.950
[7, 12000] loss: 0.970
[8,  2000] loss: 0.863
[8,  4000] 